<a href="https://colab.research.google.com/github/vivarium-collective/biosimulator-processes/blob/main/ArchiveEditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install biosimulators-utils[sbml] process-bigraph

### Upload your archive

In [2]:
from google.colab import files

In [ ]:
files.upload()

In [16]:
!pip install libsbml

ERROR: Could not find a version that satisfies the requirement libsbml (from versions: none)
ERROR: No matching distribution found for libsbml


In [83]:
# API source
import os
import tempfile
from dataclasses import dataclass, asdict
from typing import List, Tuple, Dict, Union
from process_bigraph import pp
from biosimulators_utils.combine.data_model import CombineArchive, CombineArchiveContent
from biosimulators_utils.combine.io import CombineArchiveWriter, CombineArchiveReader
from biosimulators_utils.sedml.io import SedmlSimulationReader, SedmlSimulationWriter
from biosimulators_utils.sedml.utils import get_all_sed_objects  # change this
from biosimulators_utils.sedml.model_utils import get_parameters_variables_outputs_for_simulation
from biosimulators_utils.sedml.data_model import *


# datamodel
@dataclass
class ParameterValue:
    default: Union[int, float]
    new_value: Union[int, float] = None


@dataclass
class EditableSimulationParameter:
    target: str
    value: Union[ParameterValue, Dict[str, Union[float, int]]]

    def to_dict(self):
        return asdict(self)


# exec
class ArchiveEditorApi:
    @classmethod
    def upload_archive(cls):
        return files.upload()

    @classmethod
    def get_uploaded_omex_fp(cls, root: str) -> str:
        """Get content from colab env"""
        fpaths = []
        for f in os.listdir(root):
            if '.omex' in f:
                fpaths.append(f)
        # if len(fpaths) > 1:
        #     raise OSError('You can only edit one omex archive at a time.')
        # else:
        return fpaths[0]

    @classmethod
    def read_omex(cls, omex_fp: str, out_dir: str) -> CombineArchive:
        return CombineArchiveReader().run(in_file=omex_fp, out_dir=out_dir)

    @classmethod
    def is_sedml(cls, content: CombineArchiveContent) -> bool:
        return 'simulation' in content.location

    @classmethod
    def get_sedml(cls, fp: str) -> SedDocument:
        return SedmlSimulationReader().run(filename=fp)

    @classmethod
    def get_editable_params(
            cls,
            fp: str,
            model_lang: str,
            sim_type: Simulation,
            kisao_id: str
            ) -> Tuple[List[ModelAttributeChange], List[Simulation], List[Variable], List[Plot]]:
        attrb_changes, sim, variables, plots = get_parameters_variables_outputs_for_simulation(
                           model_filename=fp,
                           model_language=model_lang,
                           simulation_type=sim_type,
                           change_level=SedDocument,
                           algorithm_kisao_id=kisao_id)
        return attrb_changes, sim, variables, plots

    @classmethod
    def parse_editable_params(cls, attributes: List[ModelAttributeChange]) -> List[EditableSimulationParameter]:
        params = []
        for attribute in attributes:
            # new_value is default and target is target
            param_value = ParameterValue(default=attribute.new_value)
            editable_param = EditableSimulationParameter(target=attribute.target, value=param_value)
            params.append(editable_param)
        return params


    @classmethod
    def run(cls, working_dir: str, colab: bool = False, kisao_id: str = None):
        if colab:
            from google.colab import files
            cls.upload_archive()

        omex_fp: str = cls.get_uploaded_omex_fp(working_dir)
        temp_extraction_dir = tempfile.mkdtemp()
        uploaded_archive: CombineArchive = cls.read_omex(omex_fp, temp_extraction_dir)

        for content in uploaded_archive.contents:
            if 'sedml' in content.location:
                sed_fp = content.location.replace('./', '')
                sed_doc: SedDocument = cls.get_sedml(
                    os.path.join(temp_extraction_dir, sed_fp))
                sed_doc_objects = get_all_sed_objects(sed_doc)
                for obj in sed_doc_objects:
                    if isinstance(obj, Task):
                        sim_type = type(obj.simulation)
                        sim_model = obj.model
                        model_lang = sim_model.language
                        model_fp = os.path.join(temp_extraction_dir, sim_model.source)

                        attributes, sim, variables, plots = cls.get_editable_params(
                            fp=model_fp,
                            model_lang=model_lang,
                            sim_type=sim_type,
                            kisao_id=kisao_id)

                        editable_params = cls.parse_editable_params(attributes)


                        pp(editable_params[0].to_dict())







ArchiveEditorApi.run('/content')

{ 'target': "/sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Trim']/@initialConcentration",
  'value': {'default': '0.084410675', 'new_value': None}}


/usr/local/lib/python3.10/dist-packages/biosimulators_utils/warnings.py:31: BioSimulatorsWarning: The SED document is potentially incorrect.
  - This tool validates XPaths for model changes and variables against their unmodified target models. As a result, this tool does not fully validate these XPaths. These XPaths could be invalidated by changes to their target models.
  - Model `Ciliberto2003_Morphogenesis` has warnings.
    - The model file `BIOMD0000000297_url.xml` has warnings.
      - 3 warnings of type SBO term consistency (10713). The following is the first warning at line 211, column 6:
        - The value of the 'sboTerm' attribute on a <species> is expected to be an SBO identifier (http://www.biomodels.net/SBO/). In SBML Level 2 prior to Version 4 it is expected to refer to a participant physical type (i.e., terms derived from SBO:0000236, "participant physical type"); in Versions 4 and above it is expected to refer to a material entity (i.e., terms derived from SBO:0000240

In [56]:
def f(*args):
    return args

In [57]:
a, b = f(2, 3)# API server

In [58]:
a# client

2

In [59]:
b

3